In [115]:
!pip install galois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 34.9 MB/s eta 0:00:00


In [116]:
import secrets as sc
import hashlib
import math
import numpy as np
import os
import galois

In [109]:
r = 7
m = 57
v = 197
shake = hashlib.shake_128()
privateSeed = sc.token_bytes(32)
field = galois.GF(2 ** 7, irreducible_poly = 0x83)

kg = KeyGenerator(r, m, v, shake, privateSeed, field)
Res = kg.KeyGen()

In [118]:
class KeyGenerator():

  def __init__(self, r, m, v, shake, privateSeed, field):
    self.r = r
    self.m = m
    self.v = v
    self.shake = shake
    self.polinomial = 0
    self.field = field
    self.n = m + v
    self.privateSeed = privateSeed
    self.publicKey = None
    self.privateKey = privateSeed

  def InitializeAndAbsorb(self, privateSeed):
    self.shake.update(privateSeed)
    return self.shake

  def SqueezePublicSeedAndT(self, privateSponge):
    hash = privateSponge.digest(32 + (self.m+7)//8*self.v)

    publicSeed = hash[:32]

    resto = hash[32:]
    bits = ''.join(format(byte, '08b') for byte in resto)
    sep = (self.m + 7) // 8 * 8
    dele = self.m % 8
    T = [bits[i:i+sep-8]+bits[i+sep-dele:i+sep]  for i in range(0, len(bits), sep)]
    T = [[ int(i) for i in j] for j in T]
    T = np.array(T, dtype = int)

    return (publicSeed, T)

  def SqueezePublicMap(self, publicSeed):
    d = self.v *(self.v +1) // 2 + self.v * self.m

    C = np.zeros((self.m, 1), dtype=int)
    L = np.zeros((self.m, self.n), dtype=int)
    Q1 = np.zeros((self.m, d), dtype=int)

    rep = (self.m + 15) // 16
    for i in range(rep):
      self.shake.update(publicSeed + bytes([i]))
      filas = self.shake.digest(2*(1 + self.n + d))

      bits = ''.join(format(byte, '08b') for byte in filas)
      bitsC = [ int(i) for i in bits[:16] ]
      bitsL = bits[16:16+2*self.n*8]
      bitsQ1 =  bits[16+2*self.n*8:]

      if i != rep-1:
        C[i*16 : (i+1)*16, 0] = bitsC

        for k in range(self.n):
          for j in range(i*16, (i+1)*16):
            L[j, k] = int(bitsL[16*k+j%16])

        for k in range(d):
          for j in range(i*16, (i+1)*16):
            Q1[j, k] = int(bitsQ1[16*k+j%16])

      else:
        C[i*16 : , 0] = bitsC[-m%16:]

        for k in range(self.n):
          for j in range(i*16, i*16+m%16):
            L[j, k] = int(bitsL[k*16+(j-m)%16])

        for k in range(d):
          for j in range(i*16, i*16+m%16):
            Q1[j, k] = int(bitsQ1[k*16+(j-m)%16])

    return (C, L, Q1)

  def find_pk1(self, k, Q1):
    pk1 = np.zeros((self.v, self.v), dtype=int)
    column = 0
    for i in range(self.v):
      for j in range(i, self.v):
        pk1[i, j] = Q1[k, column]
        column += 1
      column += self.m
    return pk1

  def find_pk2(self, k, Q1):
    pk2 = np.zeros((self.v, self.m), dtype=int)
    column = 0
    for i in range(self.v):
      column += self.v - i
      for j in range(self.m):
        pk2[i,j] = Q1[k, column]
        column += 1
    return pk2

  def find_Q2(self, Q1, T):
    D2 = self.m*(self.m+1)//2
    Q2 = np.zeros((self.m, D2), dtype=int)

    for k in range(self.m):
      Pk1 = self.find_pk1(k, Q1)
      Pk2 = self.find_pk2(k, Q1)
      Pk3 = (-T.T@Pk1@T + T.T@Pk2) % 2

      column = 0
      for i in range(self.m):
        Q2[k, column] = Pk3[i, i]
        column += 1
        for j in range(i+1, m):
          Q2[k, column] = Pk3[i][j] ^ Pk3[j][i]
          column += 1
    return Q2

  def PublicKey(self, publicSeed, Q2):
    bitsQ2 = ''
    for i in Q2.T:
      for j in i:
        bitsQ2 += str(j)

    bitsQ2 += (-self.m*self.m+1//2)%8*'0'
    bitsPS = ''.join(f'{byte:08b}' for byte in publicSeed)
    bitsCompletos = bitsPS+bitsQ2

    pk = b''
    for i in range(len(bitsCompletos)//8):
      pk += int(bitsCompletos[i*8:(i+1)*8], 2).to_bytes(1, byteorder='big')
    self.publicKey = pk

  def KeyGen(self, M):
    privateSponge = self.InitializeAndAbsorb(self.privateSeed)
    publicSeed, T = self.SqueezePublicSeedAndT(privateSponge)
    publicSponge = self.InitializeAndAbsorb(publicSeed)
    C, L, Q1 = self.SqueezePublicMap(publicSeed)
    Q2 = self.find_Q2(Q1, T)
    self.PublicKey(publicSeed, Q2)
    '''
    salt = os.urandom(16)
    h = self.genH(M, salt)

    sw = False
    sp = None
    c = 0
    '''


  def genH(self, M, salt):
    h = np.zeros((self.m, 1), dtype=int)

    seed = M + b'\x00' + salt
    self.shake.update(seed)
    hash = self.shake.digest((self.m*self.r + 7)//8)
    bits = ''.join(f'{byte:08b}' for byte in hash)

    for i in range(self.m):
      h[i, 0] = int(bits[i*self.r: (i+1)*self.r], 2)
    return h

  def BuildAugmentedMatrix(self, C, L, Q1, T, h, V):
      v_padded = self.field(np.vstack((V.reshape(-1, 1), np.zeros((self.m, 1), dtype=int))))
      v_padded_T = v_padded.T

      V = self.field(V)
      L = self.field(L)
      h = self.field(h)
      C = self.field(C)
      T  = self.field(T)

      Lv = L.dot(v_padded_T.T)
      RHS = h - C - Lv

      concat_matrix = np.vstack((-T, np.eye(self.m, dtype=int)))
      LHS = (L.dot(concat_matrix))

      for k in range(self.m):
        Pk_1 = self.find_pk1(k, Q1)
        Pk_2 = self.find_pk2(k, Q1)

        Pk_1 = self.field(Pk_1)
        Pk_2 = self.field(Pk_2)

        RHS[k] = (RHS[k] - v.T @ Pk_1 @ v)

        Fk_2 = (-(Pk_1 + Pk_1.T) @ T + Pk_2)

        LHS[k] = (LHS[k] + v @ Fk_2)

      augmented_matrix = np.hstack((LHS, RHS))
      augmented_matrix = self.field(augmented_matrix)

      return augmented_matrix

  def build_signature(self, T, v, o, sp) -> np.ndarray:

      T = self.field(T)
      v = self.field(v)
      o = self.field(o)

      identity_v = np.eye(len(v), dtype=int)
      identity_v = self.field(identity_v)

      identity_m = np.eye(len(o), dtype=int)
      identity_m = self.field(identity_m)

      top_block = np.hstack((identity_v, -T))

      bottom_block = np.hstack((np.zeros((len(o), len(v)), dtype=int), identity_m))

      block_matrix = np.vstack((top_block, bottom_block))

      s = (block_matrix @ sp)

      return s